TODO:

This is for prototyping

I will turn this into a example_action_head.py which loads the StepsDataset then trains the MLP and saves the weights

In [48]:
from datasets import load_from_disk, Dataset
import stable_worldmodel as swm
from stable_worldmodel.data import StepsDataset
from stable_worldmodel.policy import AutoCostModel

import torch
from torch import nn
import stable_pretraining as spt

In [45]:
cache_dir = "~/.stable_worldmodel"  # should be default

train_dir = "pusht_expert_dataset_train"
val_dir = "pusht_expert_dataset_val"

In [ ]:
train_path = f'{cache_dir}/{train_dir}/'
val_path   = f'{cache_dir}/{val_dir}/'

ds_train = load_from_disk(train_path)
ds_val   = load_from_disk(val_path)
# ds_train = Dataset.from_file(f"{train_dir}/data-00000-of-00001.arrow")
# ds_val   = Dataset.from_file(f"{val_dir}/data-00000-of-00001.arrow")

In [26]:
print(ds_train.features)
print(ds_train[0])

{'episode_idx': Value('int64'), 'step_idx': Value('int64'), 'action': List(Value('float64')), 'state': List(Value('float64')), 'proprio': List(Value('float64')), 'pixels': Value('string')}
{'episode_idx': 0, 'step_idx': 0, 'action': [0.14000000059604645, -0.25], 'state': [423.0, 182.0, 192.6706085205078, 335.1567687988281, 2.947699785232544], 'proprio': [423.0, 182.0, 0.0, 0.0], 'pixels': 'pusht_expert_dataset_train/img/0/0_pixels.jpeg'}


In [41]:
NUM_STEPS = 1
train_data = StepsDataset(train_dir, num_steps = NUM_STEPS)
val_data = StepsDataset(val_dir, num_steps = NUM_STEPS)

# BUG FIX -> arrow shard pointed paths wrong
for df in (train_data, val_data):
    df.data_dir = df.data_dir.parent


# print(data_train.infer_img_path_columns())

# ERROR -> stable-pretraining uses load_dataset, not load_from_disk
# FIX -> update stable-pretraining and datasets

01:33:35.063 | WARNING (10074, stable_pretraining.data.datasets) | You didn't pass a storage optionwe are adding one to avoid timeout
01:33:35.064 | INFO    (10074, stable_pretraining.data.datasets) | Loading dataset with load_from_disk /Users/ashton/.stable_worldmodel/pusht_expert_dataset_train
01:33:44.416 | WARNING (10074, stable_pretraining.data.datasets) | You didn't pass a storage optionwe are adding one to avoid timeout
01:33:44.417 | INFO    (10074, stable_pretraining.data.datasets) | Loading dataset with load_from_disk /Users/ashton/.stable_worldmodel/pusht_expert_dataset_val


In [34]:
def step_transform():
    transforms = []
    for t in range(NUM_STEPS):
        key = f"pixels.{t}"
        transforms.append(
            spt.data.transforms.Compose(
                spt.data.transforms.ToImage(
                    mean=[0.5, 0.5, 0.5],
                    std=[0.5, 0.5, 0.5],
                    source=key,
                    target=key,
                ),
                spt.data.transforms.Resize(224, source=key, target=key),
                spt.data.transforms.CenterCrop(224, source=key, target=key),
            )
        )
    return spt.data.transforms.Compose(*transforms)

transform = step_transform()
train_data = StepsDataset("pusht_expert_dataset_train", num_steps=1, transform=transform)
val_data   = StepsDataset("pusht_expert_dataset_val",   num_steps=1, transform=transform)

for df in (train_data, val_data):
    df.data_dir = df.data_dir.parent

01:21:19.957 | WARNING (10074, stable_pretraining.data.datasets) | You didn't pass a storage optionwe are adding one to avoid timeout
01:21:19.958 | INFO    (10074, stable_pretraining.data.datasets) | Loading dataset with load_from_disk /Users/ashton/.stable_worldmodel/pusht_expert_dataset_train
01:21:29.330 | WARNING (10074, stable_pretraining.data.datasets) | You didn't pass a storage optionwe are adding one to avoid timeout
01:21:29.331 | INFO    (10074, stable_pretraining.data.datasets) | Loading dataset with load_from_disk /Users/ashton/.stable_worldmodel/pusht_expert_dataset_val


In [35]:
# print(train_data[0])
print(train_data[0]['pixels'].shape)

torch.Size([1, 3, 224, 224])


In [ ]:
dinowm = AutoCostModel(model_name="dinowm") # no _object
# this is just cost_head?
dinowm.eval().requires_grad_(False)

# print(type(model))

/Users/ashton/.stable_worldmodel/dinowm_object.ckpt


In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cache_dir = swm.data.get_cache_dir()  

# torch load better?
checkpoint_name = "dinowm_object.ckpt"

checkpoint = cache_dir / checkpoint_name
dinowm = torch.load(checkpoint, map_location=device, weights_only=False)
dinowm = dinowm.to(device).eval()

for p in dinowm.parameters():
    p.requires_grad_(False)

In [ ]:
LATENT_DIM = dinowm.backbone.config.hidden_size
ACTION_DIM = len(train_data.dataset['action'][0])

# action = (x, y)
# print(LATENT_DIM, ACTION_DIM)

384 2


In [ ]:
# transforms

# from train/dinowm.py -> use 'pixels.0' for step size 1
# all the expert data should all be 224x224 already
transform = spt.data.transforms.Compose(
    spt.data.transforms.ToImage(mean=[0.5, 0.5, 0.5],
                                std=[0.5, 0.5, 0.5],
                                source="pixels.0",
                                target="pixels.0"),
    spt.data.transforms.Resize(224, source="pixels.0", target="pixels.0"),
    spt.data.transforms.CenterCrop(224, source="pixels.0", target="pixels.0"),
)

def batch_transform(batch):
    batch["pixels"] = torch.stack([transform({"pixels.0": img})["pixels.0"] for img in batch["pixels"]])
    return batch

In [ ]:
info_dict = {'pixels': batch_pixels, }

In [ ]:
EMBED_DIM = 64
ACTION_DIM = 32
EXPANSION_FACTOR = 2


# simple action head, MLP
# upproject one time
# can make take cfgs later

class ActionHead(nn.Module):
    def __init__(self, dim, hidden_dim):
        self.layers = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, dim),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.layers(x)
    
# From DINO-WM implementation:

# class FeedForward(nn.Module):
#     def __init__(self, dim, hidden_dim, dropout=0.0):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.LayerNorm(dim),
#             nn.Linear(dim, hidden_dim),
#             nn.GELU(),
#             nn.Dropout(dropout),
#             nn.Linear(hidden_dim, dim),
#             nn.Dropout(dropout),
#         )

#     def forward(self, x):
#         return self.net(x)
